# Trovares xGT

Dokumentacja: http://docs.trovares.com/v1.4/index.html

## Pierwsze połączenie

Warsztaty rozpoczniemy od zaimportowania potrzebnych modułów języka Python:
 - `xgt` - klient Trovares xGT, 
 - `pandas` - popularana biblioteka do przetwarzania danych,
 - `re` - biblioteka do przetwarzania wyrażeń regularnych (regex).

Skorzystamy także z dwóch następujących modułów pomocniczych:
 - `getpass` - narzędzia do wprowadzania haseł przez użytkowników,
 - `os`- interfejs do systemu operacyjnego.

In [ ]:
# importowanie bibliotek

import xgt
import getpass
import os

Następnie stworzymy obiekt `xgt.Connection`, który stanowi interfejs użytkownika do serwera Trovares xGT. W poniższym wywołaniu `xgt.Connection(..)`należy zmienić `kdmszkXX` na nazwę własnego użytkownika. Po wykonaniu poniższej komórki użytkownik zostanie poproszony o podanie hasła do swojego konta.

In [ ]:
# tworzenie polaczenia z serwerem
# nalezy zmienic kdmszkXX na wlasna nazwe uzytkownika

server = xgt.Connection(userid='kdmszkXX', credentials=getpass.getpass(), port=os.getuid()+20000)

Aby sprawdzić połączenie z serwerem możemy wywołać funkcje sprawdzające wersję klienta i wersję serwera. Obie powinny mieć numery `v1.4.X`.

In [ ]:
print('Wersja klienta: {}'.format(xgt.__version__))
print('Wersja serwera: {}'.format(server.server_version))

## Reprezentacja danych w Trovares xGT

Dane załadowane do Trovares xGT przechowywane są w tzw. `ramkach` (ang. `frame`). Ramki dzielą się na 3 typy: `VertexFrame`, `EdgeFrame` i `TableFrame`. Pierwszy typ przechowuje węzły grafu, drugi typ przechowuje krawędzie grafu a trzeci typ przechowuje pośrednie wyniki zapytań na dwóch pierwszych typach. Ramki grupowane są logicznie w przestrzenie nazw (ang. `namespaces`). Nazwy ramek muszą być następującej postaci: `<przestrzenNazw>__<nazwaRamki>`. W tych warsztatach nie będziemywchodzić w szczegóły techniczne przestrzeni nazw. Należy jedynie pamiętać o wspomnianej konwencji nazewniczej ramek.

Ramki tworzy się na następujące sposoby:
 - `VertexFrame` - wywołanie `xgt.Connection.create_vertex_frame(name, schema, key)`, 
 - `EdgeFrame` - wywołanie `xgt.Connection.create_edge_frame(name, schema, source, target, source_key, target_key)`
 - `TableFrame` - wywołanie zapytania na `VertexFrame` lub `EdgeFrame` w języku Trovares Query Language.

Opis parametrów do utworzenia `VertexFrame`:
 - `name` - identyfikator kwalifikowany ramki ("kwalifikowany" oznacza, że `name` musi mieć prefiks postaci `<przestrzenNazw>__`), typ: `string`
 - `schema` - schemat ramki; każda ramka ma z góry określony typ kolumn, którego nie można zmienić 'dynamicznie' (silne typowanie pól ramki), typ: `list`
 - `key` - nazwa kolumny klucza ramki z węzłami

Opis parametrów do utworzenia `EdgeFrame`:
 - `name` - identyfikator kwalifikowany ramki ("kwalifikowany" oznacza, że `name` musi mieć prefiks postaci `<przestrzenNazw>__`), typ: `string`
 - `schema` - schemat ramki; każda ramka ma z góry określony typ kolumn, którego nie można zmienić 'dynamicznie' (silne typowanie kolumn ramek), typ: lista par postaci `[key, xgt.TYPE]`, gdzie `key` to nazwa kolumny, a `xgt.TYPE` to jeden zdostępnych w Trovares xGT typów kolumn: `xgt.BOOLEAN`, `xgt.INT`, `xgt.FLOAT`, `xgt.DATE`, `xgt.TIME`, `xgt.DATETIME`, `xgt.IPADDRESS`, `xgt.TEXT`.
 - `source` - identyfikator ramki źródłowej, typ: `string`
 - `target` - identyfikator ramki docelowej, typ: `string`
 - `source_key` - nazwa kolumny klucza w ramce źródłowej, typ: `string`
 - `target_key` - nazwa kolumny klucza w ramce docelowej, typ: `string`

Więcej o sposobie tworzenia `TableVertex` znajduje się w dalej.

## Dane do przetwarzania

Podczas warsztatów będziemy korzystać ze zbioru danych `LDBC Social Network Benchmark` (przydatne linki: [strona generatora danych](https://github.com/ldbc/ldbc_snb_datagen), [specyfikacja LDBC Social Network Benchmark](http://ldbc.github.io/ldbc_snb_docs/ldbc-snb-specification.pdf)). Jest to zbiór wygenerowanych danych o fikcyjnych forach internetowych, ich użytkownikach, postach, które wysyłają itd. Więcej informacji na slajdach.

## Wczytywanie danych - wierzchołki

Wczytywanie danych zostanie zademonstrowane na przykładzie encji `Person` ze zbioru danych `LDBC Social Network Benchmark`.

![Person](obrazy/LDBC.Person.png)

[Źródło obrazka - specyfikacja LDBC Social Network Benchmark, strona 13](https://ldbc.github.io/ldbc_snb_docs/ldbc-snb-specification.pdf).

Pierwszym krokiem do wczytania danych jest upewnienie się, że docelowy `frame` jest pusty. Dzięki temu będziemy mieć pewność, że nie wystąpią błędy z powtarzającymi się identyfikatorami węzłów.

Aby upewnić się, że żadna ramka nie istnieje na serwerze skorzystamy z funkcji pomocniczej `clear_all()`, która usuwa z serwera xGT wszystkie istniejące ramki.

In [ ]:
def clear_all():
    [server.drop_frame(frame) for frame in server.get_edge_frames()]
    [server.drop_frame(frame) for frame in server.get_table_frames()]
    [server.drop_frame(frame) for frame in server.get_vertex_frames()]
    
clear_all()

Następnie możemy odtworzyć ramkę przez wywołanie funkcji `xgt.Connection.create_vertex_frame(name, schema, key)`. Dla przypomnienia - argumenty tej funkcji to:
 - `name` - nazwa nowo tworzonej ramki,
 - `schema` - lista list postaci \[`nazwa_kolumny`, `xgt.TYPE`\], definiuje typy poszczególnych kolumn,
 - `key` - nazwa kolumny z `schema`, która będzie identyfikatorem (kluczem głównym) każdego węzła.

In [ ]:
persons = server.create_vertex_frame(
        name='ldbc__persons',
        schema=[['creationDate', xgt.DATETIME], ['id', xgt.INT], ['firstName', xgt.TEXT], ['lastName', xgt.TEXT], ['gender', xgt.TEXT], ['birthday', xgt.DATE],
               ['locationIP', xgt.IPADDRESS], ['browserUsed', xgt.TEXT]],
        key='id')

Upewnijmy się jaki schemat ma ramka `persons`. Można to zrobić przez sprawdzenie pola `xgt.VertexFrame.schema`.

In [ ]:
persons.schema

Na koniec wstawimy dane do serwera Trovares xGT (należe zmienić `kdmszkXX` na nazwę własnego użytkownika).

In [ ]:
persons.load("/home/kdmszkXX/workshop/social_network/person_0_0.csv", headerMode=xgt.HeaderMode.IGNORE)

Sprawdźmy ile wierszy wczytano:

In [ ]:
persons.num_rows

Sprawdźmy pierwszych 10 węzłów jakie wczytano. Służy do tego funkcja `xgt.VertexFrame.get_data(offset, length)`.

In [ ]:
[str(person) for person in persons.get_data(length=10)]

## Obsługa błędów

Z każdą ramką skojarzony jest `TableFrame` przechowujące wszelkie błędy związane z daną ramką. Jeśli podczas wykonania kodu na danej ramce (np. przy wczytywaniu danych) pojawi się błąd, to należy sprawdzić ramkę błędów skojarzoną z ramką, na której kod wywołujemy. Poniższa komenda zwraca nazwę ramki błędów skojarzoną z `persons`.

In [ ]:
persons.error_frame_name

Poniższa komenda zwróci ramkę błędów, związaną z ramką `persons`.

In [ ]:
try:
    errors = server.get_table_frame(persons.error_frame_name)
    print("Znaleziono ramke z bledami.")
except xgt.XgtNameError as e:
    print("Nie znaleziono ramki z bledami. Oznacza to, ze nie bylo bledow podczas wczytywania danych do ramki 'persons'.")

Jeśli znaleziono ramkę z błędami to wypiszmy jej  schemat oraz pierwsze pięć błędów, które ona zawiera.

In [ ]:
try:
    print("Schemat ramki z bledami: " + str(errors.schema))
    [print(error) for error in errors.get_data(length=5)]
except (xgt.XgtNameError, NameError) as e:
    print("Nie znaleziono ramki z bledami. Nie wystapily bledy.")

Jeśli powyższy kod zwrócił jakieś błędy to należy:
1. naprawić błędy,
2. **własnoręcznie** opróżnić ramkę błędów (np. przez `xgt.Connection.drop_frame(ramka)`) - w przeciwnym razie przy kolejnych próbach manipulacji ramką oryginalną będą wyświetlane stare błędy,
3. spróbować wykonać operację na oryginalnej ramce jeszcze raz.

Poniższa komenda służy do własnoręcznego opróżniania ramki błędów.

In [ ]:
server.drop_frame(errors)

## Wczytywanie danych - krawędzie

Wczytywanie krawędzi zostanie zademonstrowane na przykładzie relacji `Person knows Person` ze zbioru `LDBC Social Network Benchmark`.

![Person knows person](obrazy/LDBC.Person_knows_Person.png)

[Źródło obrazka - specyfikacja LDBC Social Network Benchmark, strona 13](https://ldbc.github.io/ldbc_snb_docs/ldbc-snb-specification.pdf).

W analogiczny sposób do wierzchołków wczytywane są krawędzie. Służy do tego funkcja `xgt.Connection.create_edge_frame(..)` (sekcja `Reprezentacja danych w Trovares xGT`, patrz wyżej).



In [ ]:
person_knows_person = server.create_edge_frame(
    name="ldbc__person_knows_person",
    schema=[['creationDate', xgt.DATETIME], ['Person1.id', xgt.INT], ['Person2.id', xgt.INT]],
    source=persons,
    target=persons,
    source_key='Person1.id', 
    target_key='Person2.id')

Sprawdźmy schemat ramki `person_knows_person`.

In [ ]:
person_knows_person.schema

Teraz możemy załadować dane do ramki `person_knows_person` (należy zmienić `kdmszkXX` na nazwę własnego użytkownika).

In [ ]:
person_knows_person.load("/home/kdmszkXX/workshop/social_network/person_knows_person_0_0.csv", headerMode=xgt.HeaderMode.IGNORE)

Sprawdźmy ile krawędzi wczytano.

In [ ]:
person_knows_person.num_rows

Sprawdźmy pierwsze 10 krawędzi jakie wczytano.

In [ ]:
[str(knows) for knows in person_knows_person.get_data(length=10)]

## Zadania do samodzielnej pracy

 1. Wczytaj wszystkie wierzchołki typu`Forum` z pliku `social_network/forum_0_0.csv`.
 2. Wczytaj krawędzie o moderatorach for internetowych, tzn. krawędzie typu `HasModerator` z wierzchołków typu `Forum` do wierzchołków typu `Person` z pliku `social_network/forum_hasModerator_person_0_0.csv`.

In [ ]:
# miejsce na odpowiedz
